
# Spot resources Analytics

Here we perform some initial process and analysis on the dataset.

---




With static dataset, e.g. load the grabbed data.

In [ ]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
%matplotlib inline


# parse the data file and extra the results
filename = 'data1'

df = pd.read_csv(filename, sep="\t", header = None)
df.columns = ["info", "SpotPrice", "TimeStamp", "InstanceType", "OS type", "AvailabilityZone"]
df['TimeStamp'] =pd.to_datetime(df.TimeStamp)

df.index = df.TimeStamp
df = df.drop('info', 1).drop(['OS type'],axis=1)
df = df.drop(['TimeStamp'],axis=1).sort_index()
 

df.head(15)

In [ ]:
print (df['InstanceType'].unique())
print (df['AvailabilityZone'].unique())

In [ ]:
instance_types  = ['c3.xlarge', 'c3.2xlarge', 'c3.4xlarge', 'c3.8xlarge']
region = 'us-east-1'

df1 = df[df.AvailabilityZone == 'us-west-1a']
df2 = df1[df1.InstanceType == 'c3.8xlarge']
df2.to_csv('us-east-1a_c3-8xlarge.csv')

In [ ]:
for k, g in df1.sort_index(ascending=True).groupby(['InstanceType'], as_index=False):
    for key, grp in g.groupby(['AvailabilityZone'], as_index=False):
        plt.figure(figsize=(15,5))
        plt.plot(grp.index, grp['SpotPrice'], label=key)
        
    plt.legend()
    plt.title('Spot Pricing - ' + k)
    plt.show()


In [ ]:
for k, g in df1.sort_index(ascending=True).groupby(['InstanceType'], as_index=False):
    #plt.figure(1, figsize(20,5))
    for key, grp in g.groupby(['AvailabilityZone'], as_index=False):
        plt.figure(figsize=(15,5))
        plt.hist(grp['SpotPrice'], bins=100, label=key,)
        plt.xlim([0, 1])
        #grp.groupby(grp.index.dayofweek).agg(['mean']).plot()
    plt.legend()
    plt.title('Histogram of Spot Pricing - ' + k)
    plt.show()

Now we grad dataset from AWS.

In [ ]:
instance_types  = ['c3.xlarge', 'c3.2xlarge', 'c3.4xlarge', 'c3.8xlarge']
region = 'us-east-1'
number_of_days = 10

end = !date -u "+%Y-%m-%dT%H:%M:%S"
end = end[0]
start = !date -v-{number_of_days}d -u "+%Y-%m-%dT%H:%M:%S"

#start = !date -v-{number_of_days}d" -u "+%Y-%m-%dT%H:%M:%S"
#print(start)
start = start[0]
print ("will process from " + start + " to " + end)


In [ ]:
import sys
import boto as boto
import boto.ec2 as ec2
import datetime, time
#import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.mpl_style', 'default')  # Make the graphs a bit prettier
%pylab inline
%matplotlib inline

ec2 = boto.ec2.connect_to_region(region)


#
# process the output and convert to a dataframe
#

l = []
for instance in instance_types:
    sys.stdout.write("*** processing " + instance + " ***\n")
    sys.stdout.flush()
    prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance)
    for price in prices:
        d = {'InstanceType': price.instance_type, 
             'AvailabilityZone': price.availability_zone, 
             'SpotPrice': price.price, 
             'Timestamp': price.timestamp}
        l.append(d)
    next = prices.next_token
    while (next != ''):
        sys.stdout.write(".")
        sys.stdout.flush()
        prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance,
                                            next_token=next )
        for price in prices:
            d = {'InstanceType': price.instance_type, 
                 'AvailabilityZone': price.availability_zone, 
                 'SpotPrice': price.price, 
                 'Timestamp': price.timestamp}
            l.append(d)
        next = prices.next_token
        
    sys.stdout.write("\n")

df = pd.DataFrame(l)
df = df.set_index(pd.to_datetime(df['Timestamp']))

## Hypothesis #1
**Problems:** Can we predict future price of a spot instance given previous history and how other vm’s are reacting?

To achieve the goal of prediction, we are expecting to do pattern matching from the collected dataset. In this case, whenever users make a bid, we can based on the resources types, time or day, and the trending price to do pattern matching. We will be able to provide a prediction if we can shoot a pattern.

Expecting tools are supervised and unsupervised learning algorithms, e.g. classification and
clustering methods.



## Hypothesis #2

For each machine type there exists a region that is more favorable to use, as the market volatility is very low and the prices tend to stay cheaper than the other regions.

With in proving this hypothesis users will be able to find the best region they should be bidding in, as long as latency is not an issue for them.

Data Science tools & Techniques: We can use clustering and classification methods.


In [ ]:
print (df.index.min())
print (df.index.max())
print(df.index.max()- df.index.min()) 
#df = df.truncate(before='2015-01-16', after='2015-3-18')

In [ ]:
df = df.resample('H')
df = df.fillna("ffill")

In [ ]:
#Create full time series and fill data
dfSorted = df.groupby(['AvailabilityZone', 'InstanceType'])
dfSorted = dfSorted.resample('H')
dfSorted = dfSorted.fillna("ffill")

dfSorted=dfSorted.drop('InstanceType', axis=1).drop('AvailabilityZone', axis=1)

dfSorted.to_csv("im.csv")
depa = pd.read_csv("im.csv")
depa = depa.groupby(['AvailabilityZone', 'InstanceType'])


In [ ]:
#grouped_prices = [group['SpotPrice'].tolist() for name, group in depa]

#dfer = dataframe
d = {}

count = 0
#need to run through and get rid of rows where timestamp, spotprice data doesnt exist for the others
for name, group in depa:
    if count == 0:
        d['TimeStamp']=group['TimeStamp'].tolist()
    if name[0] =="ap-northeast-1a":
        for a in d['TimeStamp']+group['TimeStamp'].tolist():
            if(a not in d['TimeStamp']):
                group = group[group['TimeStamp'] != a]
                print (a)
            if(a not in group['TimeStamp'].tolist()):
                d['TimeStamp'].remove(a)
                print (a)
    
    
    #seter = set(d['TimeStamp']) - set(group['TimeStamp'].tolist())
    #print(seter)
    #remove = list(seter)
    #print(remove)    
#dfer = pd.DataFrame(d)
    
#for name, group in depa:
     #print(len(group['SpotPrice'].tolist()))
        
'''
for name, group in depa:
    if name[0] =="ap-northeast-1a":
        #group.index = group['TimeStamp']
        #print(group.head(20))
        #group = group.truncate(before='2015-01-18', after='2015-3-17')
        d[name[1]]=group['SpotPrice'].tolist()
        print(len(group['SpotPrice'].tolist()))
'''

In [ ]:
#numpy.corrcoef(grouped_prices)

grouped_prices.corr()
'''
mask = np.zeros_like(corr_df)
mask[np.triu_indices_from(mask)] = True
seaborn.heatmap(corr_df, cmap='RdYlGn_r', vmax=1.0, vmin=-1.0 , mask = mask, linewidths=2.5)
# Show the plot we reorient the labels for each column and row to make them easier to read.
plt.yticks(rotation=0) 
plt.xticks(rotation=90) 
plt.show()
'''


# Hypothesis #3

There exists some kind of relation between what kind of virtual machines are turning into hotspots. Say that we establish a line as half price of EC2 instances, it makes sense to pay half price to gain usage of resources but probably not more than 3⁄4. By extracting patterns from the price history, we can study that whether or not there was the case that some resources were becoming hotspot in the spot instances market.

Potential data science method for this one includes: Time Series, Linear Regression


In [ ]:
df = pd.read_csv('us-east-1a_c3-8xlarge.csv')
#df.head(400)
df = df2
df.describe()

df.SpotPrice.plot(label='Spot Price of c3.8xlarge', figsize = (15,5))
plt.legend(loc='best')

In [ ]:
df.head(15)

# Dataset pre-process

Some preprocess work


In [47]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
%matplotlib inline


# parse the data file and extra the results
filename = 'aws-spot-price-history/data-1397804701'

df = pd.read_csv(filename, sep="\t", header = None)
df.columns = ["info", "SpotPrice", "TimeStamp", "InstanceType", "OS type", "AvailabilityZone"]
df['TimeStamp'] =pd.to_datetime(df.TimeStamp)

df.index = df.TimeStamp
df = df.drop(['TimeStamp'],axis=1).drop('info', 1).sort_index()


df.head(10)
#df['AvailabilityZone']
#df['InstanceType']

,SpotPrice,InstanceType,OS type,AvailabilityZone
TimeStamp,,,,
2014-02-16 00:54:00,0.0100,m1.small,Linux/UNIX,us-west-1a
2014-02-16 00:55:30,2.7320,c3.8xlarge,Linux/UNIX,us-west-1a
2014-02-16 00:58:09,0.0679,m3.large,Linux/UNIX,us-west-1b
2014-02-16 00:59:56,3.0240,c3.8xlarge,Linux/UNIX,ap-southeast-2a
2014-02-16 01:17:30,0.0224,m3.medium,Linux/UNIX,us-east-1d
2014-02-16 01:17:37,0.0100,m1.small,Linux/UNIX,us-west-1b
2014-02-16 01:24:19,0.7561,c3.2xlarge,Linux/UNIX,ap-southeast-2b
2014-02-16 01:24:19,0.7561,c3.2xlarge,Linux/UNIX,ap-southeast-2a
2014-02-16 01:24:25,0.0926,m3.xlarge,Linux/UNIX,ap-southeast-2b


In [54]:
region_list = ["us-west-1a","us-west-1b" ]
vm_types_list = ["c3.large", "c3.xlarge", "c3.2xlarge","c3.4xlarge", "c3.8xlarge"]

dicts = {}

for r in region_list:
    r = {}
    for t in vm_types_list:
        #print(t)
        df1 = df[df.AvailabilityZone == r]
        df2 = df1[df1.InstanceType == t]
        #print(df2.head(5))
        # get the time series data
        r[t] = df2["SpotPrice"].tolist()

print(len())

                     SpotPrice InstanceType     OS type AvailabilityZone
TimeStamp                                                               
2014-02-16 21:37:34        2.0     c3.large  Linux/UNIX       us-west-1a
2014-02-17 21:38:26        2.0     c3.large  Linux/UNIX       us-west-1a
2014-02-18 21:38:35        2.0     c3.large  Linux/UNIX       us-west-1a
2014-02-19 21:39:40        2.0     c3.large  Linux/UNIX       us-west-1a
2014-02-20 21:40:04        2.0     c3.large  Linux/UNIX       us-west-1a
                     SpotPrice InstanceType     OS type AvailabilityZone
TimeStamp                                                               
2014-02-16 18:13:46     0.3421    c3.xlarge  Linux/UNIX       us-west-1a
2014-02-17 18:13:48     0.3421    c3.xlarge  Linux/UNIX       us-west-1a
2014-02-18 18:14:32     0.3421    c3.xlarge  Linux/UNIX       us-west-1a
2014-02-19 18:15:16     0.3421    c3.xlarge  Linux/UNIX       us-west-1a
2014-02-20 18:15:20     0.3421    c3.xlarge  Linux/

In [44]:
"""
different_regions = [

    df_us_west_one_a = df[df.AvailabilityZone == "us-west-1a"],
    df_us_west_one_b = df[df.AvailabilityZone == "us-west-1b"],

    df_us_east_one_a = df[df.AvailabilityZone == "us-east-1a"],
    df_us_east_one_b = df[df.AvailabilityZone == "us-east-1b"],
    df_us_east_one_c = df[df.AvailabilityZone == "us-east-1c"],
    df_us_east_one_d = df[df.AvailabilityZone == "us-east-1d"],

    df_ap_southeast_one_a = df[df.AvailabilityZone == "ap-southeast-1a"],
    df_ap_southeast_one_b = df[df.AvailabilityZone == "ap-southeast-1b"],

    df_ap_southeast_two_a = df[df.AvailabilityZone == "ap-southeast-2a"],
    df_ap_southeast_two_b = df[df.AvailabilityZone == "ap-southeast-2b"]

    df[df.AvailabilityZone == "us-west-1a"],
    df[df.AvailabilityZone == "us-west-1b"],

    df[df.AvailabilityZone == "us-east-1a"],
    df[df.AvailabilityZone == "us-east-1b"],
    df[df.AvailabilityZone == "us-east-1c"],
    df[df.AvailabilityZone == "us-east-1d"],

    df[df.AvailabilityZone == "ap-southeast-1a"],
    df[df.AvailabilityZone == "ap-southeast-1b"],

    df[df.AvailabilityZone == "ap-southeast-2a"],
    df[df.AvailabilityZone == "ap-southeast-2b"]
]
"""

df_us_west_one_a = df[df.AvailabilityZone == "us-west-1a"]
df_us_west_one_b = df[df.AvailabilityZone == "us-west-1b"]

df_us_east_one_a = df[df.AvailabilityZone == "us-east-1a"]
df_us_east_one_b = df[df.AvailabilityZone == "us-east-1b"]
df_us_east_one_c = df[df.AvailabilityZone == "us-east-1c"]
df_us_east_one_d = df[df.AvailabilityZone == "us-east-1d"]

df_ap_southeast_one_a = df[df.AvailabilityZone == "ap-southeast-1a"]
df_ap_southeast_one_b = df[df.AvailabilityZone == "ap-southeast-1b"]

df_ap_southeast_two_a = df[df.AvailabilityZone == "ap-southeast-2a"]
df_ap_southeast_two_b = df[df.AvailabilityZone == "ap-southeast-2b"]

#c3,c3_x,c3_2x,c3_4x,c3_8x = [],[],[],[],[]
#c3_8x = {}
#for dataframe in df_us_west_one_a,df_us_west_one_b,df_ap_southeast_two_a,df_ap_southeast_two_b, \
    #df_us_east_one_a,df_us_east_one_b,df_us_east_one_c,df_us_east_one_d,\
    #df_ap_southeast_one_a, df_ap_southeast_one_b:
    
    #df2 = dataframe[dataframe.InstanceType == 'c3.8xlarge']
    #dflist = df2["SpotPrice"]
    #c3_8x.append(dflist)
    #c3_8x["1"] = dflist
#print(c3_8x)


c3_8x = {}
for dff in df_us_west_one_a,df_us_west_one_b,df_ap_southeast_two_a,df_ap_southeast_two_b, \
    df_us_east_one_a,df_us_east_one_b,df_us_east_one_c,df_us_east_one_d,\
    df_ap_southeast_one_a, df_ap_southeast_one_b:
    
    df2 = df[df.InstanceType == 'c3.8xlarge']
    dflist = df2["SpotPrice"]
    
    
    dataframe = df_us_west_one_a
df2 = dataframe[dataframe.InstanceType == 'c3.8xlarge']
dflist = df2["SpotPrice"]
c3_8x["us_west_one_a"] = dflist

dataframe = df_us_west_one_b
df2 = dataframe[dataframe.InstanceType == 'c3.8xlarge']
dflist = df2["SpotPrice"]
c3_8x["us_west_one_b"] = dflist

print(len(c3_8x))
print(c3_8x["us_west_one_a"])


#variations = [dflist_x.tolist()[:1000], dflist_2x.tolist()[:1000], dflist_4x.tolist()[:1000], dflist_8x.tolist()[:1000]]
"""
instance_types  = ['c3.xlarge', 'c3.2xlarge', 'c3.4xlarge', 'c3.8xlarge']
region1 = 'us-east-1'

df1 = df[df.AvailabilityZone == 'us-west-1a']
df2 = df1[df1.InstanceType == 'c3.8xlarge']
df2.to_csv('us-east-1a_c3-8xlarge.csv')
dflist_8x = df2["SpotPrice"]


df2 = df1[df1.InstanceType == 'c3.4xlarge']
df2.to_csv('us-east-1a_c3-4xlarge.csv')
dflist_4x = df2["SpotPrice"]


df2 = df1[df1.InstanceType == 'c3.2xlarge']
df2.to_csv('us-east-1a_c3-2xlarge.csv')
dflist_2x = df2["SpotPrice"]


df2 = df1[df1.InstanceType == 'c3.xlarge']
df2.to_csv('us-east-1a_c3-xlarge.csv')
dflist_x = df2["SpotPrice"]



for i in instance_types:
    print(i)
    df1 = df[df.AvailabilityZone == region1]
    df2 = df1[df1.InstanceType == i]
    #df2.to_csv(i)
"""
#df2.head(20)
#df2["SpotPrice"]
#df_ap_southeast_two_b.head(10)


us-west-1a
c3.large
c3.xlarge
c3.2xlarge
c3.4xlarge
c3.8xlarge
us-west-1b
c3.large
c3.xlarge
c3.2xlarge
c3.4xlarge
c3.8xlarge
2
TimeStamp
2014-02-16 00:55:30    2.7320
2014-02-17 00:56:30    2.7320
2014-02-18 00:57:12    2.7320
2014-02-19 00:57:56    2.7320
2014-02-20 00:58:42    2.7320
2014-02-21 00:58:59    2.7320
2014-02-21 07:59:55    3.0000
2014-02-21 08:59:47    2.7321
2014-02-22 09:00:21    2.7321
2014-02-23 09:01:02    2.7321
2014-02-24 09:01:29    2.7321
2014-02-25 09:02:15    2.7321
2014-02-25 19:00:49    0.4991
2014-02-25 19:02:11    0.4073
2014-02-25 20:31:27    0.4088
2014-02-25 22:02:16    0.4077
2014-02-25 22:56:06    0.4087
2014-02-26 00:02:24    0.4077
2014-02-26 00:32:48    0.4087
2014-02-26 01:43:26    0.4076
2014-02-26 05:10:18    0.4100
2014-02-26 06:19:01    0.4089
2014-02-26 08:04:32    0.4155
2014-02-26 08:42:55    0.4277
2014-02-26 09:06:20    0.4295
2014-02-26 09:09:05    0.4340
2014-02-26 09:11:51    0.4316
2014-02-26 09:14:37    0.4295
2014-02-26 09:15:59   

'\ninstance_types  = [\'c3.xlarge\', \'c3.2xlarge\', \'c3.4xlarge\', \'c3.8xlarge\']\nregion1 = \'us-east-1\'\n\ndf1 = df[df.AvailabilityZone == \'us-west-1a\']\ndf2 = df1[df1.InstanceType == \'c3.8xlarge\']\ndf2.to_csv(\'us-east-1a_c3-8xlarge.csv\')\ndflist_8x = df2["SpotPrice"]\n\n\ndf2 = df1[df1.InstanceType == \'c3.4xlarge\']\ndf2.to_csv(\'us-east-1a_c3-4xlarge.csv\')\ndflist_4x = df2["SpotPrice"]\n\n\ndf2 = df1[df1.InstanceType == \'c3.2xlarge\']\ndf2.to_csv(\'us-east-1a_c3-2xlarge.csv\')\ndflist_2x = df2["SpotPrice"]\n\n\ndf2 = df1[df1.InstanceType == \'c3.xlarge\']\ndf2.to_csv(\'us-east-1a_c3-xlarge.csv\')\ndflist_x = df2["SpotPrice"]\n\n\n\nfor i in instance_types:\n    print(i)\n    df1 = df[df.AvailabilityZone == region1]\n    df2 = df1[df1.InstanceType == i]\n    #df2.to_csv(i)\n'

In [ ]:
"""
for k, g in df1.sort_index(ascending=True).groupby(['InstanceType'], as_index=False):
    for key, grp in g.groupby(['AvailabilityZone'], as_index=False):
        plt.figure(figsize=(15,5))
        plt.plot(grp.index, grp['SpotPrice'], label=key)
        
    plt.legend()
    plt.title('Spot Pricing - ' + k)
    plt.show()

for k, g in df1.sort_index(ascending=True).groupby(['InstanceType'], as_index=False):
    #plt.figure(1, figsize(20,5))
    for key, grp in g.groupby(['AvailabilityZone'], as_index=False):
        plt.figure(figsize=(15,5))
        plt.hist(grp['SpotPrice'], bins=100, label=key,)
        plt.xlim([0, 1])
        #grp.groupby(grp.index.dayofweek).agg(['mean']).plot()
    plt.legend()
    plt.title('Histogram of Spot Pricing - ' + k)
    plt.show()
"""

# Time Series Clustering

## Time series clustering on spot market price data set.

Here we will be using TSC to analyse relations between various of types of machines and find out relationships from our clustering results.

The first step is to work out an appropriate distance/similarity metric. Secondly, we will use existing clustering techniques, such as k-means, hierarchical clustering, density-based clustering or subspace clustering, to find clustering structures.

Dynamic Time Warping (DTW) finds optimal alignment between two time series, and DTW distance is used as a distance metric in the example below.




A data set of Synthetic Control Chart Time Series is used here, which contains 600 examples of control charts. Each control chart is a time series with 60 values. There are six classes: 1) 1-100 Normal, 2) 101-200 Cyclic, 3) 201-300 Increasing trend, 4)301-400 Decreasing trend, 5) 401-500 Upward shift, and 6) 501-600 Downward shift. The dataset is downloadable at UCI KDD Archive.

In [9]:
#print(__doc__)

# Author: Gael Varoquaux gael.varoquaux@normalesup.org
# License: BSD 3 clause

import datetime

import numpy as np
import matplotlib.pyplot as plt
try:
    from matplotlib.finance import quotes_historical_yahoo_ochl
except ImportError:
    # quotes_historical_yahoo_ochl was named quotes_historical_yahoo before matplotlib 1.4
    from matplotlib.finance import quotes_historical_yahoo as quotes_historical_yahoo_ochl
from matplotlib.collections import LineCollection
from sklearn import cluster, covariance, manifold

In [ ]:
# Choose a time period reasonably calm (not too long ago so that we get
# high-tech firms, and before the 2008 crash)
"""

d1 = datetime.datetime(2003, 1, 1)
d2 = datetime.datetime(2008, 1, 1)

symbols, names = np.array(list(symbol_dict.items())).T
#print(symbols, names)

quotes = [quotes_historical_yahoo_ochl(symbol, d1, d2, asobject=True)
          for symbol in symbols]

open = np.array([q.open for q in quotes]).astype(np.float)
close = np.array([q.close for q in quotes]).astype(np.float)

# The daily variations of the quotes are what carry most information
variation = close - open

#print(variation)
print(len(variation))
type(variation)
"""

In [10]:
#list_x = dflist_x.tolist()

variations = [dflist_x.tolist()[:1000], dflist_2x.tolist()[:1000], dflist_4x.tolist()[:1000], dflist_8x.tolist()[:1000]]
#print(variations)
#prices = np.array([q.open for q in quotes]).astype(np.float)
#print(len(variations[0]))
prices = np.array(variations)

print(prices)
#prices.astype(np.float)

[[ 0.3421  0.3421  0.3421 ...,  0.0519  0.052   0.0521]
 [ 0.6831  0.6831  1.     ...,  0.1519  0.1384  0.1238]
 [ 0.4     0.4     0.45   ...,  0.1287  0.1288  0.129 ]
 [ 2.732   2.732   2.732  ...,  0.4064  0.4063  0.4064]]


In [11]:
edge_model = covariance.GraphLassoCV()

# standardize the time series: using correlations rather than covariance
# is more efficient for structure recovery
#X = variation.copy().T
X = prices.copy().T
print(type(X))
#print(X)

#X /= X.std(axis=0)
X = X / X.std(axis=0)

edge_model.fit(X)

<class 'numpy.ndarray'>


GraphLassoCV(alphas=4, assume_centered=False, cv=None, enet_tol=0.0001,
       max_iter=100, mode='cd', n_jobs=1, n_refinements=4, tol=0.0001,
       verbose=False)

In [12]:
_, labels = cluster.affinity_propagation(edge_model.covariance_)
n_labels = labels.max()

for i in range(n_labels + 1):
    print('Cluster %i: %s' % ((i + 1), ', '.join(names[labels == i])))

NameError: name 'names' is not defined

In [ ]:
# We use a dense eigen_solver to achieve reproducibility (arpack is
# initiated with random vectors that we don't control). In addition, we
# use a large number of neighbors to capture the large-scale structure.
node_position_model = manifold.LocallyLinearEmbedding(
    n_components=2, eigen_solver='dense', n_neighbors=1)

embedding = node_position_model.fit_transform(X.T).T

In [ ]:
plt.figure(1, facecolor='w', figsize=(10, 8))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')

# Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)

# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)
#a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)

# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):

    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.spectral(label / float(n_labels)),
                       alpha=.6))

plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())

plt.show()

# TS Clustering Try #2

In [ ]:
import numpy as np;
import seaborn as sns;
import pandas as pd
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt

num_samples = 10
group_size = 4

x = np.linspace(0, 15, num_samples)
a = np.sin(x) + np.linspace(0, 5, num_samples)

x = np.linspace(0, 50, num_samples)
b = np.sin(x) + np.linspace(0, -8, num_samples)
c = np.sin(x + 2)

d = np.linspace(0, 14, num_samples)
e = np.random.randn(group_size, 1) + np.linspace(0, -3, num_samples)

x = np.linspace(0, 4, num_samples)
f = np.sin(x)

variances = [dflist_x.tolist()[:1000], dflist_2x.tolist()[:1000], dflist_4x.tolist()[:1000], dflist_8x.tolist()[:1000]]


timeSeries = pd.DataFrame()
ax = None

#for arr in [a,b,c,d,e,f]:
for arr in [dflist_x.tolist()[:1000], dflist_2x.tolist()[:1000], dflist_4x.tolist()[:1000], dflist_8x.tolist()[:1000]]:    
    #print(arr)
    print(np.random.rand(group_size, num_samples))
    print((np.random.randn(group_size, 1)*3))
    #arr = arr + np.random.rand(group_size, num_samples) + (np.random.randn(group_size, 1)*3)
    #print(arr)
    df = pd.DataFrame(arr)
    timeSeries = timeSeries.append(df)

    # We use seaborn to plot what we have
    #ax = sns.tsplot(ax=ax, data=df.values, ci=[68, 95])
    #ax = sns.tsplot(ax=ax, data=df.values, err_style="unit_traces")

In [ ]:
# Just one line :)
Z = hac.linkage(timeSeries, 'single', 'correlation')

# Plot the dendogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
hac.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
# Here we decided to use spearman correlation
correlation_matrix = timeSeries.T.corr(method='spearman')

# Do the clustering
Z = hac.linkage(correlation_matrix, 'single')

# Plot dendogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
hac.dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

### Reference:
1. [](http://scikit-learn.org/stable/modules/generated/sklearn.covariance.GraphLassoCV.html)
2. [](http://scikit-learn.org/stable/modules/covariance.html)


1. [Mining Time-series with Trillions of Points: Dynamic Time Warping at scale](http://practicalquant.blogspot.com/2012/10/mining-time-series-with-trillions-of.html)
2. [sackoverflow post](http://stackoverflow.com/questions/34940808/hierarchical-clustering-of-time-series-in-python-scipy-numpy-pandas)
3. [scilearn material](http://scikit-learn.org/stable/modules/clustering.html)
4. [pattern matching over time series data](http://stats.stackexchange.com/questions/136091/sequential-pattern-matching-in-time-series-data)

http://sklearn.lzjqsdd.com/modules/covariance.html